In [3]:
# Modules
import gurobipy as grb
import pandas as pd
import numpy as np
import math
import random

# Standard output divider
div = "-----"

# Print warning message


# Timeslots
alltimeslots = {
    1: "8:00am",
    2: "8:45am",
    3: "9:30am",
    4: "10:30am",
    5: "11:15am",
    6: "12:00pm",
    7: "1:15pm",
    8: "2:00pm",
    9: "2:45pm",
    10: "3:45pm",
    11: "4:30pm",
    12: "5:15pm",
    13: "6:30pm",
    14: "7:15pm",
    15: "8:00pm",
    16: "8:45pm"
}



# Dictionary of preference strings and corresponding values
preferencedict = {
    "Available and Prefer": 1,
    "Available": 2,
    "Available but Prefer Not": 3,
    "Unavailable": 4
}

# Dictionary of speaker vetos and corresponding values

vetodict = {
    "I would not like to veto any position": 0,
    "1st Speaker": 1,
    "2nd Speaker": 2,
    "3rd Speaker": 3,
}

vetodictinv = {
    0: "has no speaker position vetoes",
    1: "has vetoed 1st speaker",
    2: "has vetoed 2nd speaker",
    3: "has vetoed 3rd speaker"
}

# Person class
# - No preferences or vetoes for Easters trials

class person:
    name = ""
    
    # Speaker veto - 1 for 1st, 2 for 2nd, 3 for 3rd, 0 for no veto
    speakerveto = 0
    
    # Key-value pair for availability - Key is the slot
    # 1 = Prefer
    # 2 = Available
    # 3 = Prefer Not
    # 4 = Unavailable
    availabilities = {}
    
    def __init__(self,name,availabilities,speakerveto,isAllocated):
        self.name = name
        self.availabilities = availabilities
        self.speakerveto = speakerveto
        isAllocated = False
        
        
def importData():
    print("Please note the final allocation may take up to 10 minutes to compute.")
    speakers = []
    times = {}
    # CSV reading and parsing
    csv = pd.read_csv("Easters 2019 Trial Registration (Responses) - Responses.csv")
    for index,row in csv.iterrows():
        if row[7] == "Debating only" or row[7] == "Debating, and if I am not put in a team, adjudicating":
            avail = {}
            for i in range(10,26):
                avail[i-9] = preferencedict[row[i]]
            speakers.append(person(row[1],avail,vetodict[row[18]],False))

    # Calculate how many timeslots are required
    reqd = int(np.floor(len(speakers)/6))
    for key,value in alltimeslots.iteritems():
        if key <= reqd:
            times[key] = value
    print("A minimum of {} debates is required to distribute {} speakers.".format(reqd,len(speakers)))
    
    '''If the allocator produces an infeasible result there's a good chance it needs more timeslots.
    Manually override times with a higher number of slots if required'''
    
    #times = alltimeslots[:x]

    # Form the feasible team set
    ftset = {}
    ftsample = {}
    totalteams = len(speakers)**3
    index = 1
    print("Forming Feasible Team Set...")
    for speaker1 in speakers:
        for speaker2 in speakers:
            for speaker3 in speakers:
                if speaker1 != speaker2 and speaker1 != speaker3 and speaker2 != speaker3:
                    if speaker1.speakerveto != 1 and speaker2.speakerveto != 2 and speaker3.speakerveto != 3:
                        print("Constructing team {} of {}...".format(index,totalteams))
                        ftset[index] = [speaker1,speaker2,speaker3]
                        index += 1
    
    randomkeys = random.sample(ftset.keys(), len(ftset)/5)
    for k in randomkeys:
        ftsample[k] = ftset[k]
    
    # Costs
    costs = {}
    totalcosts = len(ftsample)*len(times)
    print("Calculating costs...")
    index2 = 1
    for key,team in ftsample.iteritems():
        for slot in times.keys():
            costs[key,slot] = 0
            index2 += 1
            print("Calculating cost {} of {}...".format(index2,totalcosts))
    
        # Time preferences
        '''Triallists are made aware that they may be called during "unavailable",
        so an 'unavailable' does not make the cost infinitely high, it just incurs
        a high cost.
        
        As slots become less desirable, the cost increases exponentially rather than
        linearly'''
    
        for speaker in team:
            for slot in times.keys():
                mag = speaker.availabilities[slot]*100
                exp = mag**2
                costs[key,slot] += mag
    
def allocate():
    # Optimisation
    a = grb.Model("2019 Easters Trial Allocator")
    x = {}
    for key,value in costs.iteritems():
        x[key] = a.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(key))
        
    # Exactly 2 teams to each timeslot
    for slot in times.keys():
        a.addConstr(sum(x[team,slot] for team in fts.keys()) == 2)
        
    # Each team to no more than 1 timeslot
    for key,team in ftsample.iteritems():
        a.addConstr(sum(x[key,slot] for slot in times.keys()) <= 1)

    thirdvetoes = []
    for s in speakers:
        affectedTeams = []
        for key,team in ftsample.iteritems():
            if s in team:
                affectedTeams.append(key)
                
        # Each speaker to no more than 1 debate in 1 timeslot
        '''Can be 0 debates (ie. unallocated) and then appended
        as alternate 3rds'''
        a.addConstr(sum(x[team,slot] for team in affectedTeams for slot in times.keys()) <= 1)
        
        if s.speakerveto == 3:
            thirdvetoes.append(s)
            
    # Speakers who veto 3rd must be in exactly 1 timeslot
    '''3rd vetoes cannot be unallocated and then tacked on as
    alternate 3rds, so they must be allocated in a debate.'''
    for speaker in thirdvetoes:
        affectedTeams2 = []
        for key,team in ftsample.iteritems():
            if speaker in team:
                affectedTeams2.append(key)
        a.addConstr(sum(x[team,slot] for team in affectedTeams2 for slot in times.keys()) == 1)
    
    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
    
    unallocated = []
    unallocatednames = []
    print(div)
    print("USU EASTERS TRIALS 2019 - DEBATE ALLOCATION")
    print("Speakers in each team are listed in speaking order.")
    print("Sides for each debate should be determined randomly.")
    print(div)
    for key,value in costs.iteritems():
        if x[key].x != 0:
            spk = []
            for s in ftsample[key[0]]:
                spk.append(s.name)
            print("Team {} {} at {}".format(key[0],spk,times[key[1]]))
            for speaker in ftsample[key[0]]:
                speaker.isAllocated = True
    for speaker in speakers:
        if speaker.isAllocated == False:
            unallocated.append(speaker)
            unallocatednames.append(speaker.name)
    print(div)
    
    j = ", "
    print("Unallocated Speakers: {}".format(j.join(unallocatednames)))
    print(div)
    report(unallocated)

def report(speakers):
    for speaker in speakers:
        if speaker not in speakers:
            print("Speaker {} does not exist".format(speaker.name))
        else:
            vetoString= "{} {}.".format(speaker.name, vetodictinv[speaker.speakerveto])

            print("Report for {}".format(speaker.name))
            print(vetoString)
            print("{}'s availability is as follows:".format(speaker.name))
            for slot,availability in speaker.availabilities.iteritems():
                try:
                    # TODO
                    pass
                    #print("{}: {}".format(times[slot],availability))
                except:
                    pass
        print(div)

In [4]:
importData()

Please note the final allocation may take up to 10 minutes to compute.
A minimum of 0 debates is required to distribute 0 speakers.
Forming Feasible Team Set...
Calculating costs...


In [ ]:
allocate()